In [1]:
# Install required libraries
#pip install pdf2image pytesseract opencv-python ollama pandas

     ---------------------------------------- 66.7/66.7 MB 3.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 45.5/45.5 kB 2.4 MB/s eta 0:00:00
Failed to build llama-cpp-python


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [20 lines of output]
  *** scikit-build-core 0.10.7 using CMake 3.31.4 (wheel)
  *** Configuring CMake...
  2025-02-04 23:13:00,516 - scikit_build_core - WARNING - Can't find a Python library, got libdir=None, ldlibrary=None, multiarch=None, masd=None
  loading initial cache file C:\Users\melod\AppData\Local\Temp\tmpwsx0pt38\build\CMakeInit.txt
  -- Building for: NMake Makefiles
  CMake Error at CMakeLists.txt:3 (project):
    Running
  
     'nmake' '-?'
  
    failed with:
  
     no such file or directory
  
  
  CMake Error: CMAKE_C_COMPILER not set, after EnableLanguage
  CMake Error: CMAKE_CXX_COMPILER not set, after EnableLanguage
  -- Configuring incomplete, errors occurred!
  
  *** CMake configuration failed
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for llama-cpp-python
ERROR: Could not build wheels for llama-cpp-python,

In [12]:
import pytesseract
from pdf2image import convert_from_path
import cv2
import numpy as np
import pandas as pd
import json
import ollama
from IPython.display import display, Markdown
import ipywidgets as widgets
import os

In [13]:
# Convert PDF to images
def pdf_to_images(pdf_path):
    return convert_from_path(pdf_path, dpi=300)

In [14]:
# Extract text using Tesseract OCR
def extract_text_from_images(images):
    extracted_text = ""
    for img in images:
        text = pytesseract.image_to_string(img)
        extracted_text += text + "\n"
    return extracted_text

In [15]:
# Extract tables using OpenCV & Tesseract
def extract_tables_from_images(images):
    tables = []
    for img in images:
        gray = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        table_text = pytesseract.image_to_string(thresh, config="--psm 6")

        # Convert text into structured table format
        table_lines = [line.split() for line in table_text.split("\n") if line.strip()]
        if table_lines:
            df = pd.DataFrame(table_lines)
            tables.append(df)

    return tables

In [16]:
# Process PDF and extract structured data
def process_pdf(pdf_path):
    images = pdf_to_images(pdf_path)
    text = extract_text_from_images(images)
    tables = extract_tables_from_images(images)
    return {"text": text, "tables": tables}

In [23]:
#Identification Layer
def identLayer(text, table_data, table_item):
    prompt = f"""
    You are an AI analyst that extracts insights by reasoning over structured data.

    Extracted Information From The Document is:
    {text}

    Extracted Table Data:
    {json.dumps([table.to_dict() for table in table_data], indent=2)}

    Fetch table line item "{table_item}" and identify its trend over the specified period in the document.
    """

    response = ollama.chat(model="llama3", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

In [24]:
def getIdentLayerResponse(pdf_path, table_item):
    data = process_pdf(pdf_path)
    response = infoExtLayer(data["text"], data["tables"], table_item)
    return response

In [25]:
#Information Extraction Layer
def infoExtLayer(text, table_data, table_item):
    prompt = f"""
    You are an AI analyst that extracts insights by reasoning over structured data.

    Extracted Information From The Document is:
    {text}

    Extracted Table Data:
    {json.dumps([table.to_dict() for table in table_data], indent=2)}

    Focus only on the specific table line item: "{table_item}".
    - Identify this item in the Income Statement Summary table.
    - Collect all the related information related to "{table_item}" in the extracted text.
    - Provide all the structured facts and insights from the document and respond concisely.
    """

    response = ollama.chat(model="llama3", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

In [26]:
def getInfoExtLayerResponse(pdf_path, table_item):
    data = process_pdf(pdf_path)
    response = infoExtLayer(data["text"], data["tables"], table_item)
    return response

In [27]:
#Reasoning/Implication Layer
def reasLayer(text, table_item):
    prompt = f"""
    You are an AI analyst that extracts insights by reasoning over structured data.

    Given below are the facts and insights gathered from the given document about the line item {table_item}.
    {text}

    Focus only on the specific table line item: "{table_item}" and using the factors that affect it :
    - Identify the factors affecting {table_item} and correlate a few references of those factors from the text.
    - Pick up cause-effect relationships from the text corresponding to the factors present in the rules.
    - Collect a set of implications or reasons that attributed to the observed change in {table_item}.
    - List them concisely with quoted reference to statistical data and informational evidence from the text.
    - Reasoning to be confined only to the content of the pdf and its statistical evidences. Any information having 
    """

    response = ollama.chat(model="llama3", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

In [21]:
def getReasLayerResponse(rules, text, table_item):
    response = reasLayer(rules, text, table_item)
    return response

In [ ]:
trend = getIdentLayerResponse("gxo.pdf", "Revenue")

In [ ]:
layer1_response = getInfoExtLayerResponse("gxo.pdf", trend)

In [1]:
getReasLayerResponse(layer1_response, trend)


The increase in revenue is primarily attributed to the following factors:  

1. **Acquisitions**  
   - The **Clipper Acquisition** contributed **$378 million** in revenue for periods that were not comparable year-over-year.  
   - The **PFS Acquisition** added **$82 million** to the total revenue.  

2. **Geographic Growth**  
   - Revenue growth in **Continental Europe** played a significant role in the overall increase.  

3. **Foreign Currency Movements**  
   - Favorable foreign exchange fluctuations contributed **$140 million** to the revenue increase.  



In [ ]:
trend = getIdentLayerResponse("gxo.pdf", "Operating Income")

In [ ]:
layer1_response = getInfoExtLayerResponse("gxo.pdf", trend)

In [2]:
getReasLayerResponse(layer1_response, trend)


The increase in **Operating Income** is primarily due to the following factors:  

### **Factors Contributing to the Increase in Operating Income:**  
1. **Revenue Growth**  
   - Operating income increased as a result of **higher revenue**, driven by the Clipper and PFS acquisitions, as well as growth in Continental Europe.  

2. **Lower Transaction and Integration Costs**  
   - Transaction and integration costs **decreased by $27 million**, from **$61 million in 2022 to $34 million in 2023**. The reduction was due to lower costs associated with integrating Clipper and PFS acquisitions.  

3. **Higher Selling, General, and Administrative (SG&A) Expenses**  
   - SG&A expenses increased **by 13% ($112 million)** due to **higher personnel costs** and **bad debt expenses**. However, this was offset by business growth.  

4. **Increase in Depreciation and Amortization**  
   - Depreciation and amortization expenses rose **by $32 million (10%)**, primarily due to the **Clipper acquisitio

In [ ]:
trend = getIdentLayerResponse("gxo.pdf", "Net Income")

In [ ]:
layer1_response = getInfoExtLayerResponse("gxo.pdf", trend)

In [3]:
getReasLayerResponse(layer1_response, trend)


The increase in **Net Income** is primarily due to the following factors:  

### **Factors Contributing to the Increase in Net Income:**  
1. **Revenue Growth**  
   - Increased revenue from the **Clipper and PFS acquisitions** and **growth in Continental Europe** contributed to higher net income.  

2. **Lower Transaction and Integration Costs**  
   - These costs decreased **by $27 million**, reducing overall expenses.  

3. **Lower Effective Tax Rate**  
   - The **effective tax rate decreased from 24.2% in 2022 to 12.4% in 2023**, mainly due to tax benefits from intangible assets and the release of valuation allowances. This resulted in a **$31 million reduction in income tax expense**.  

### **Factors Offsetting the Increase in Net Income:**  
1. **Higher Interest Expense**  
   - Interest expense **increased by $24 million (83%)** due to the Clipper acquisition debt being **outstanding for a full year** and rising variable interest rates.  

2. **Lower Other Income**  
   - Oth

In [27]:
#Inductive Reasoning/Implication Layer
def reasLayerInductive(text, table_item):
    prompt = f"""
    You are an AI analyst that extracts insights by reasoning over structured data.

    Given below are the facts and insights gathered from the given document about the line item {table_item}.
    {text}

    Focus only on the specific table line item: "{table_item}" and using the factors that affect it :
    - Identify the factors affecting {table_item} and correlate a few references of those factors from the text.
    - Pick up cause-effect relationships from the text corresponding to the factors present in the rules.
    - Collect a set of implications or reasons that attributed to the observed change in {table_item}.
    - List them concisely with quoted reference to statistical data and informational evidence from the text.
    - Reasoning to be confined only to the content of the pdf and its statistical evidences. Any information having 
    """

    response = ollama.chat(model="llama3", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

In [21]:
def getReasLayerInductiveResponse(rules, text, table_item):
    response = reasLayer(rules, text, table_item)
    return response

In [ ]:
trend = getIdentLayerResponse("gxo.pdf", "Net Income")

In [ ]:
layer1_response = getInfoExtLayerResponse("gxo.pdf", trend)

In [5]:
getReasLayerResponse(layer1_response, trend)


Here’s an analysis for the **Net Income** change based on the document:  

### **Analysis of Net Income**  

1. **Pattern in Revenue Growth and Acquisitions**  
   - Revenue increased by **9% ($785 million)** in 2023, primarily due to the **Clipper** and **PFS** acquisitions.  
   - Acquisitions contributed a **combined $460 million** to revenue, suggesting a **trend where strategic acquisitions drive financial growth**.  

2. **Trend in Tax Benefits**  
   - The **effective tax rate dropped from 24.2% to 12.4%**, resulting in a **$31 million reduction in income tax expense**.  
   - This suggests a **potential pattern where tax optimization strategies contribute to net income growth** across fiscal years.  

3. **Interest Expense and Debt Patterns**  
   - Interest expense **increased by 83% ($24 million)** due to debt from the **Clipper acquisition and rising interest rates**.  
   - If this trend continues, **future net income growth could be constrained by increasing financing cos

In [ ]:
## Different reasoning techniques